In [2]:
import numpy as np
import pandas as pd
import jieba
import tensorflow

/home/light/App/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
df = pd.read_csv('../data/train.csv')

In [4]:
df.groupby('subject').describe()

sentiment_value                                             
                  count      mean       std  min  25%  50%  75%  max
subject                                                             
价格               1273.0 -0.024352  0.450581 -1.0  0.0  0.0  0.0  1.0
内饰                536.0 -0.065299  0.700753 -1.0 -1.0  0.0  0.0  1.0
动力               2732.0  0.002196  0.528218 -1.0  0.0  0.0  0.0  1.0
外观                489.0  0.008180  0.680476 -1.0  0.0  0.0  0.0  1.0
安全性               573.0  0.012216  0.580744 -1.0  0.0  0.0  0.0  1.0
操控               1036.0  0.175676  0.620135 -1.0  0.0  0.0  1.0  1.0
油耗               1082.0  0.012015  0.516914 -1.0  0.0  0.0  0.0  1.0
空间                442.0  0.196833  0.679929 -1.0  0.0  0.0  1.0  1.0
舒适性               931.0 -0.155747  0.608556 -1.0 -1.0  0.0  0.0  1.0
配置                853.0 -0.039859  0.565690 -1.0  0.0  0.0  0.0  1.0

In [5]:
df.head(2)

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高


In [6]:
oh_df = pd.get_dummies(df, columns=['subject'])

In [7]:
oh_df.head(2)

,content_id,content,sentiment_value,sentiment_word,subject_价格,subject_内饰,subject_动力,subject_外观,subject_安全性,subject_操控,subject_油耗,subject_空间,subject_舒适性,subject_配置
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,0,影响,1,0,0,0,0,0,0,0,0,0
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,-1,高,1,0,0,0,0,0,0,0,0,0


In [8]:
theme = '价格'

In [9]:
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stops = set([v.strip() for v in f.readlines()])
    return stops
stops = load_stopwords('../data/stopwords.txt')
def seg(text):
    result = [word for word in jieba.cut(text) if word not in stops]
    return result

In [10]:
def count_words(count_dict, serial):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in serial:
        for word in seg(sentence):
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [11]:
word_counts = {}
count_words(word_counts, oh_df['content'])
            
print("Size of Vocabulary:", len(word_counts))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.815 seconds.
Prefix dict has been built succesfully.


Size of Vocabulary: 16280


In [14]:
def load_embeddings(embedding_file, header=True):
    # Load embeddings
    embeddings_index = {}
    with open(embedding_file) as f:
        if header:
            print(f.readline())
        for line in f:
            line = line.strip()
            if not line:
                break
            try:
                word, vec = line.split(' ', 1)
            except:
                print(line)
            vec = [float(v) for v in vec.split(' ')]
            embedding = np.asarray(vec, dtype='float32')
            embeddings_index[word] = embedding

    print('Word embeddings:', len(embeddings_index))
    return embeddings_index


In [13]:
def save_embedding(word_embedding_index, path):
    first = True
    with open(path, 'w') as f:
        for k, rep in word_embedding_index.items():
            line = '{} {}'.format(k, ' '.join([str(v) for v in rep]))
            if first:
                first = False
            else:
                f.write('\n')
            f.write(line)
        

In [15]:
embeddings_index = load_embeddings('../data/cliped_vec.vec', header=False)

/n


AttributeError: 'list' object has no attribute 'split'

In [ ]:
def vocab2int2vocab(word_counts, embeddings_index):
    # Limit the vocab that we will use to words that appear ≥ threshold or are in embedding

    #dictionary to convert words to integers
    vocab_to_int = {} 

    value = 0
    for word, count in word_counts.items():
        if word in embeddings_index:
            vocab_to_int[word] = value
            value += 1

    # Special tokens that will be added to our vocab
    codes = ["<UNK>","<PAD>"]   

    # Add codes to vocab
    for code in codes:
        vocab_to_int[code] = len(vocab_to_int)

    # Dictionary to convert integers to words
    int_to_vocab = {}
    for word, value in vocab_to_int.items():
        int_to_vocab[value] = word

    usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

    print("Total number of unique words:", len(word_counts))
    print("Number of words we will use:", len(vocab_to_int))
    print("Percent of words we will use: {}%".format(usage_ratio))
    return vocab_to_int, int_to_vocab

In [ ]:
vocab2int, int2vocab = vocab2int2vocab(word_counts, embeddings_index)

In [ ]:
def get_embedding_matrix(vocab_to_int, embeddings_index):
    # Need to use 300 for embedding dimensions to match CN's vectors.
    embedding_dim = 300
    nb_words = len(vocab_to_int)
    not_in = 0
    # Create matrix with default values of zero
    word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
    for word, i in vocab_to_int.items():
        if word in embeddings_index:
            word_embedding_matrix[i] = embeddings_index[word]
        elif word == '<PAD>':
            not_in += 1
            new_embedding = np.zeros((embedding_dim, ))
            embeddings_index[word] = new_embedding
            word_embedding_matrix[i] = new_embedding
        else:
            not_in += 1
            # If word not in CN, create a random embedding for it
            new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
            embeddings_index[word] = new_embedding
            word_embedding_matrix[i] = new_embedding

    # Check if value matches len(vocab_to_int)
    print(len(word_embedding_matrix))
    print('not in:', not_in)
    return word_embedding_matrix

In [ ]:
embedding_matrix = get_embedding_matrix(vocab2int, embeddings_index)
save_embedding(embeddings_index, '../data/filled.es.vec')

In [ ]:
def convert_to_ints(serial, vocab_to_int, word_count, unk_count):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in serial:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [ ]:
def get_labels(df, theme):
    theme_col = 'subject_{}'.format(theme)
    for row in df.iterrows():
        value = [0, 0, 0, 0]
        if row[theme_col] == 0:
            key = 0
        else:
            key = int(row['sentiment_value'] + 2)
        value[key] = 1
        yield value

In [ ]:
word_count = 0
unk_count = 0

int_es, word_count, unk_count = convert_to_ints(oh_df['content'], vocab2int, word_count, unk_count)
label = list(oh_df, theme)

In [ ]:
def pad_sentence_batch(sentence_batch, vocab_to_int, max_sentence=40):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    return [sentence[:max_sentence] + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence[:max_sentence])) for sentence in sentence_batch]

In [ ]:
def split_train_val(X, Y, val_size=3000):
    indices = np.arange(len(X1))
    np.random.shuffle(indices)
    val_idxs = set(indices[:val_size])
    val_X, val_Y = [], []
    train_X, train_Y = [], []
    match_count = 0
    for idx in indices:
        if idx in val_idxs:
            val_X1.append(X[idx])
            val_Y.append(Y[idx])
            match_count += Y[idx]
        else:
            train_X.append(X[idx])
            train_Y.append(Y[idx])
    print('match rate', match_count/val_size)
    return (train_X, train_Y), (val_X, val_Y)

In [ ]:
def get_batch(X, Y=None, num_epochs=4, batch_size=10, echo=True, shuffle=True):
    
    batch_x = []
    if Y:
        batch_y = []
    indices = np.arange(len(X))
    for epoch in range(num_epochs):
        if echo:
            print('-----------epoch:', epoch)
        if shuffle:
            np.random.shuffle(indices)
        for idx in indices:
            x = X[idx]
            if Y:
                y = Y[idx]
            batch_x.append(x)
            if Y:
                batch_y.append(y)
            if len(batch_x1) == batch_size:
                bx = pad_sentence_batch(batch_x, es_vocab2int)
                if Y:
                    yield bx, batch_y
                else:
                    yield bx
                batch_x = []
                if Y:
                    batch_y = []
        

In [ ]:
from textcnn import TextCNN

In [ ]:
(train_X, train_Y), (val_X, val_Y) = split_train_val(int_es, label, val_size=300)

In [ ]:
checkpoint = './textcnn.ckpt'
work_dir = 'my_graph'
test_size = 1000
test_loss = 10000

stop_early = 0
stop = 40

print_step = 3
update_step = 10

with tf.Graph().as_default() as g:
    cnn = TextCNN(seq_len=80, embedding=es_embedding_matrix, filter_sizes=[3, 4, 5], num_filters=256, num_classes=4, hidden_units=128, l2_reg=0)
    cnn.create_placeholder()
    cnn.create_variable()
    cnn.create_model()
    cnn.create_loss(lr=0.0001)
    with tf.Session(graph=g) as sess:
        writer = tf.summary.FileWriter(work_dir, sess.graph)
        
        # global_step = tf.Variable(0, name='global_step', trainable=False)
        sess.run(tf.global_variables_initializer())

        step = 0
        for batch_x, batch_y in get_batch(train_X, train_Y, num_epochs=8, batch_size=30):
            feed_dict = {
                cnn.x: batch_x,
                cnn.y: batch_y,
                cnn.keep_prob: 0.5,
            }
            # loss_summary = tf.summary.scalar("loss", cnn.loss)
            # loss_smry, loss, _ = sess.run([loss_summary, cnn.loss, cnn.train_op], feed_dict=feed_dict)
            loss, _ = sess.run([cnn.loss, cnn.train_op], feed_dict=feed_dict)
            step += 1
            
            # writer.add_summary(loss_smry, step)
           
            
            if step % print_step == 0:
                print('step:', step, ', loss:', loss)
            if step % update_step == 0:
                val_loss = []
                for batch_x, batch_y in get_batch(val_X, val_Y, num_epochs=1, batch_size=20, echo=False):
                    feed_dict = {
                        cnn.x: batch_x,
                        cnn.y: batch_y,
                        cnn.keep_prob: 1,
                    }
                    loss, _ = sess.run([cnn.loss, cnn.train_op], feed_dict=feed_dict)
                    val_loss.append(loss)
                update_loss = sum(val_loss)/len(val_loss)
                print('test loss:', update_loss)
                if update_loss <= test_loss:
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)
                    test_loss = update_loss
                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0

                
